In [1]:
import torch
from transformers import GPT2LMHeadModel
from transformers import PreTrainedTokenizerFast

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import re
from tqdm import tqdm
import urllib.request

In [2]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK) 
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [3]:
# urllib.request.urlretrieve(
#     "https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv",
#     filename="ChatBotData.csv",
# )
Chatbot_Data = pd.read_csv("ChatBotData.csv") # 일상다반사 0, 이별(부정) 1, 사랑(긍정) 2로 레이블링
Chatbot_Data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [4]:
text = pd.read_csv('./data/새 폴더/KETI_대화데이터_일상_오피스.txt', delimiter='	', header=None)
text1 = text[text[0] == 1].drop(0, axis=1)
text2 = text[text[0] == 2].drop(0, axis=1)
text1.columns = ['Q']
text2.columns = ['A']
text = pd.concat([text1.reset_index(drop=True),text2.reset_index(drop=True)], axis=1)
text['label'] = 0
text

,Q,A,label
0,좋은 아침.,안녕하세요.,0
1,좋은 아침.,반갑습니다.,0
2,좋은 아침.,좋은 아침이에요.,0
3,좋은 아침.,간밤에 별일 없으셨죠?,0
4,좋은 아침.,안녕하시렵니까?,0
...,...,...,...
1320,눈 오는것 같은데?,"네, 눈길 조심하세요.",0
1321,너는 오늘 기분 어때?,기분 좋은 하루에요.,0
1322,무슨 생각해?,아무 생각도 안하는걸요.,0
1323,넌 뭐야?,잘 모르겠어요.,0


In [5]:
Chatbot_Data = pd.concat([Chatbot_Data, text], axis=0)
Chatbot_Data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
1320,눈 오는것 같은데?,"네, 눈길 조심하세요.",0
1321,너는 오늘 기분 어때?,기분 좋은 하루에요.,0
1322,무슨 생각해?,아무 생각도 안하는걸요.,0
1323,넌 뭐야?,잘 모르겠어요.,0


# 2회차 데이터 추가

In [6]:
daily1 = pd.read_json('./data/새 폴더/output_ daily_1st.json')
daily2 = pd.read_json('./data/새 폴더/output_daily_2nd.json')
daily3 = pd.read_json('./data/새 폴더/output_daily_3rd.json')

In [7]:
dailys = [daily1, daily2, daily3]

In [8]:
for i in range(len(dailys)):
    dailys[i] = dailys[i][['user_utterance','system_utterance']]
    dailys[i] = dailys[i][dailys[i].user_utterance != 'null'].reset_index(drop=True)
    dailys[i].columns = ['Q', 'A']
    dailys[i]['label'] = 0
daily = pd.concat(dailys)
daily

,Q,A,label
0,알고 있어.,오늘도 고생 많이 하셨습니다.,0
1,밖에 날씨는?,월요일 퇴근전 현재 맑고 저온건조한데다가 공기는 맑은 편입니다.,0
2,알았다.,오늘도 고생이 많으셨습니다.,0
3,퇴근한다.,오늘도 일하느라 수고하셨어요.,0
4,알면 됐다.,그럼 내일 뵙겠습니다.,0
...,...,...,...
1990,그렇다.,곧 미팅이 시작됩니다. 잠시만 기다려주세요.,0
1991,그러하다.,오랜만에 오셨군요. 환영합니다.,0
1992,환영해주어서 감사해.,미팅이 곧 시작됩니다. 잠시만 기다려주세요.,0
1993,맞아.,어서오십시오. 오랜만입니다.,0


In [9]:
Chatbot_Data = pd.concat([Chatbot_Data, daily])
Chatbot_Data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [10]:
Chatbot_Data.shape

(26797, 3)

# 4회차 데이터셋 변경 및 추가(존댓말)

In [11]:
from datasets import load_dataset

ds1 = load_dataset("beomi/KoAlpaca-v1.1a", split="train")
ds2 = load_dataset("HumanF-MarkrAI/WIKI_QA_Near_dedup")
print(len(ds1))
print(len(ds2['train']))

21155
137505


In [12]:
ds1 = pd.DataFrame(ds1) # ['instruction'] ['output']
ds2 = pd.DataFrame(ds2['train'])

ds1.columns=['Q','A','label']
ds1.label = 0
ds1.head()

,Q,A,label
0,양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?,양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\...,0
1,스웨터의 유래는 어디에서 시작되었나요?,스웨터의 유래는 14세기경 북유럽항구지역에서 어망을 짜던 기술을 의복에 활용하면서 ...,0
2,토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요? \n\n토성의 고리는 얼음과 ...,"토성의 고리가 미세한 입자들로 이루어져 있기 때문에, 입자들의 밀도 차이 때문에 카...",0
3,화장품 OEM과 화장품 ODM의 차이점은 무엇인가요?\n화장품 자체 제조 브랜드 런...,화장품 제조업체는 대체로 OEM과 ODM을 통해 제품을 만듭니다. OEM은 브랜드에...,0
4,"'사이보그'는 언제 처음 등장한 말이며, 그 의미와 종류에는 어떤 것이 있는지 알고...","'사이보그'는 1960년에 처음 등장한 말로, 기계와 유기체가 합성되어 생겨난 새로...",0


In [13]:
# 엔터기호 삭제
def subtext(x):
    return re.sub('[\n]', '', x)
ds1['Q'] = ds1.Q.apply(subtext)
ds1['A'] = ds1.A.apply(subtext)
ds1.head()

,Q,A,label
0,양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?,양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. 식물의...,0
1,스웨터의 유래는 어디에서 시작되었나요?,스웨터의 유래는 14세기경 북유럽항구지역에서 어망을 짜던 기술을 의복에 활용하면서 ...,0
2,토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요? 토성의 고리는 얼음과 같은 여...,"토성의 고리가 미세한 입자들로 이루어져 있기 때문에, 입자들의 밀도 차이 때문에 카...",0
3,화장품 OEM과 화장품 ODM의 차이점은 무엇인가요?화장품 자체 제조 브랜드 런칭을...,화장품 제조업체는 대체로 OEM과 ODM을 통해 제품을 만듭니다. OEM은 브랜드에...,0
4,"'사이보그'는 언제 처음 등장한 말이며, 그 의미와 종류에는 어떤 것이 있는지 알고...","'사이보그'는 1960년에 처음 등장한 말로, 기계와 유기체가 합성되어 생겨난 새로...",0


In [14]:
ds2 = ds2.drop('input', axis=1)
ds2 = ds2[['instruction', 'output']]
ds2.columns = ['Q', 'A']
ds2['label'] = 0
ds2.head()

,Q,A,label
0,캐모마일은 어떤 식물인가요?,"캐모마일은 국화과에 속하는 식물로, 저먼캐모마일, 캐모마일, 카미레 등이 포함됩니다...",0
1,상하이 미술관은 언제 폐관되었어?,"상하이 미술관은 중화인민공화국 상하이시에 위치했던 미술관으로, 2012년 12월 3...",0
2,널로리카타는 어떤 동물 목인가?,신군부목(Neoloricata)은 다판류를 대표하는 연체동물 목 중 하나입니다. 그...,0
3,김영현은 어떤 작품들을 작성했나요?,"김영현은 대한민국의 유명한 드라마 작가로, 많은 작품들이 높은 시청률로 인기를 얻었...",0
4,클레오파트라 7세는 누구야?,클레오파트라는 고대 그리스어로 '아버지의 영광'이라는 뜻의 이름이에요. 클레오파트라...,0


In [15]:
print(ds1.shape)
print(ds2.shape)

(21155, 3)
(137505, 3)


In [16]:
Chatbot_Data = pd.concat([Chatbot_Data, ds1, ds2])
print(Chatbot_Data.shape)
Chatbot_Data.head()

(185457, 3)


,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


# 5회차 데이터 추가(존댓말 데이터)

In [17]:
import json
with open('data/aihub/ko_wiki_v1_squad.json', 'r', encoding='utf-8') as file:
    data = json.load(file)
data['data'][0]

{'paragraphs': [{'qas': [{'question': '다테 기미코가 최초로 은퇴 선언을 한게 언제지',
     'answers': [{'answer_start': 260, 'text': '1996년 9월 24일'}],
     'id': '9_f2_wiki_2822-1'}],
   'context': "재팬 오픈에서 4회 우승하였으며, 통산 단식 200승 이상을 거두었다. 1994년 생애 최초로 세계 랭킹 10위권에 진입하였다. 1992년에는 WTA로부터 '올해 가장 많은 향상을 보여준 선수상'(Most Improved Player Of The Year)을 수여받았으며, 일본 남자 패션 협회(Japan Men's Fashion Association)는 그녀를 '가장 패셔너블한 선수'(Most Fashionable)로 칭했다. 생애 두 번째 올림픽 참가 직후인 1996년 9월 24일 최초로 은퇴를 선언하였다. 이후 12년만인 2008년 4월에 예상치 못한 복귀 선언을 하고 투어에 되돌아왔다. 2008년 6월 15일 도쿄 아리아케 인터내셔널 여자 오픈에서 복귀 후 첫 우승을 기록했으며, 2009년 9월 27일에는 한국에서 열린 한솔 코리아 오픈 대회에서 우승하면서 복귀 후 첫 WTA 투어급 대회 우승을 기록했다. 한숨 좀 작작 쉬어!"}],
 'title': '다테_기미코'}

In [18]:
data['data'][100]['paragraphs'][0]['qas'][0]['answers'][0]

{'answer_start': 9, 'text': '건륭'}

In [19]:
data['data'][100]['paragraphs'][0]['qas'][0]['question']

'청나라의 7대 황제가 누구야'

In [20]:
q = []
a = []
for d in tqdm(data['data']):
    for text in d['paragraphs'][0]['qas']:
        q.append(text['question'])
        a.append(text['answers'][0]['text'])

100%|██████████| 68538/68538 [00:00<00:00, 1623358.58it/s]


In [21]:
len(q), len(a)

(100268, 100268)

In [22]:
data = pd.DataFrame([q, a]).T
data.head()

,0,1
0,다테 기미코가 최초로 은퇴 선언을 한게 언제지,1996년 9월 24일
1,ave;new 본거지 어디야,도쿄 치요다구
2,사카이 다다요시의 아버지가 누구지,사카이 나오타카(酒井直隆)
3,일반성면의 면적이 얼마야,19.41 km²
4,금나라를 세운 사람이 누구야,태조 아구다


In [23]:
data.columns = ['Q','A']
data['label'] = 0
data.head()

,Q,A,label
0,다테 기미코가 최초로 은퇴 선언을 한게 언제지,1996년 9월 24일,0
1,ave;new 본거지 어디야,도쿄 치요다구,0
2,사카이 다다요시의 아버지가 누구지,사카이 나오타카(酒井直隆),0
3,일반성면의 면적이 얼마야,19.41 km²,0
4,금나라를 세운 사람이 누구야,태조 아구다,0


In [24]:
Chatbot_Data = pd.concat([Chatbot_Data, data])
Chatbot_Data.shape

(285725, 3)

In [25]:
# 과학상식
with open('data/hugginface/science_textbook_elementary_kor_test.json', 'r', encoding='utf-8') as file:
    data1 = json.load(file)
len(data1)

1061

In [26]:
q, a = [], []
for d in data1:
    if d['input'] == '':
        q.append(d['instruction'])
        a.append(d['output'])
len(q), len(a)

(960, 960)

In [27]:
data = pd.DataFrame([q, a]).T
data.columns = ['Q','A']
data['label'] = 0
data.head()

,Q,A,label
0,물체란 무엇인가요?,"일정한 모양이 있고, 공간을 차지하는 것을 물체라고 합니다.",0
1,학교나 집에서 볼 수 있는 물체에는 어떤 것이 있을까요?,"학교에서는 책상, 의자, 연필, 책 등을 볼 수 있습니다. 집에서는 냄비, 식탁, ...",0
2,우리 주변의 물체는 여러 가지 재료로 만듭니다. 연필 몸통과 클립은 각각 어떤 재료...,연필 몸통은 나무로 만들어졌습니다. 클립은 금속으로 만들어졌습니다.,0
3,"나무, 금속, 고무, 플라스틱과 같이 물체를 만드는 재료를 무엇이라고 할까요?",물질이라고 합니다.,0
4,물질의 종류에는 어떤 것이 있는지 이야기해 볼까요?,"나무, 금속, 고무, 플라스틱, 유리, 섬유, 가죽 등이 있습니다.",0


In [28]:
Chatbot_Data = pd.concat([Chatbot_Data, data])
print(Chatbot_Data.shape)
Chatbot_Data.head()

(286685, 3)


,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [29]:
# ㅋㅋㅋ # 문제 없는 듯함

In [30]:
# Chatbot_Data.to_csv('chatbot_data_240801.csv', index=False)

### 위까지 내용은 파일로 저장함

In [31]:
import json
with open('data/aihub/감성대화말뭉치(최종데이터)_Training.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [32]:
q = []
a = []
for text in data:
    q.append(text['talk']['content']['HS01'])
    a.append(text['talk']['content']['SS01'])
    if text['talk']['content']['HS02']:
        q.append(text['talk']['content']['HS02'])
        a.append(text['talk']['content']['SS02'])
        if text['talk']['content']['HS03']:
            q.append(text['talk']['content']['HS03'])
            a.append(text['talk']['content']['SS03'])

In [33]:
len(q), len(a)

(145949, 145949)

In [34]:
data = pd.DataFrame([q, a]).T
data.columns = ['Q','A']
data['label'] = 0
data.head()

,Q,A,label
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,0
1,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.,0
2,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,0
3,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.,0
4,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,0


In [35]:
Chatbot_Data = pd.concat([Chatbot_Data, data])
print(Chatbot_Data.shape)
Chatbot_Data.head()

(432634, 3)


,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [36]:
Chatbot_Data.shape

(432634, 3)

In [37]:
zzzzzz

NameError: name 'zzzzzz' is not defined

In [38]:
Chatbot_Data.to_csv('chatbot_data_432634.csv', index=False)

## 아래는 chip2와 대화 내용을 기록한 것들 

In [ ]:
from datasets import load_dataset

ds3 = load_dataset("heegyu/OIG-small-chip2-ko")
ds3

DatasetDict({
    train: Dataset({
        features: ['user', 'chip2', 'index', 'user_translated', 'chip2_translated'],
        num_rows: 210282
    })
})

In [ ]:
ds3['train'][0]['user_translated']
ds3['train'][0]['chip2_translated']

'유언장은 귀하가 사망한 후 귀하의 재산이 어떻게 분배되어야 하는지를 지정하는 법적 문서입니다. 또한 귀하가 가질 수 있는 자녀나 기타 부양가족을 누가 돌봐야 하는지 명시할 수 있습니다. 유언장에 적용되는 법률이 주마다 다르기 때문에 귀하의 유언장이 유효하고 최신인지 확인하는 것이 중요합니다.'

# 3회차 데이터 추가

In [ ]:
# 압축파일 해제없이 내부 파일 읽기
from zipfile import ZipFile
import json
import glob
import os

all_data = []
for i in glob.glob(os.getcwd()+'\\data\\01.데이터\\1.Training\\라벨링데이터\\*.zip'):
    data_list = []
    with ZipFile(i, 'r') as zipObj:
        listOfFileNames = zipObj.namelist()
        for fileName in listOfFileNames:
            zipRead = zipObj.read(fileName)
            json_data = zipRead.decode('utf-8')
            try:
                data=[]
                z = json.loads(json_data) # 여기서 오류가 나오는 경우가 있으므로 try문 적용
                chat = z['info'][0]['annotations']
                if chat['speaker_type'] == '1:1':
                    for i in chat['lines']:
                        data.append(i['norm_text'].replace('키키','').strip())
                if len(data) > 0:
                    data_list.append(data)
            except:
                pass
    all_data.append(data_list)

In [ ]:
print(len(all_data))
print(len(all_data[0]))
print(len(all_data[1]))
print(len(all_data[0][0]))

8
12334
13338
17


In [ ]:
labeling_data = []
for j in all_data:
    df_list = []
    for i in j:
        if len(i)%2 == 1:
            i = i[:-1]
        df = pd.DataFrame([i[::2], i[1::2]]).T
        df_list.append(df)
    df_list = pd.concat(df_list)
    labeling_data.append(df_list)
labeling_data = pd.concat(labeling_data)
labeling_data.columns = ['Q','A']
labeling_data['label'] = 0

In [ ]:
labeling_data.head()

,Q,A,label
0,오빠야 저녁 식사는 뭐야?,나는 아직 뭐 먹을지 모르겠네 니는?,0
1,난 일단 지금 라면 먹고 있는데 저녁 모르겠어,오 지금 라면을 먹음 저녁 먹을 수 있나,0
2,모르겠어 이제 일어나서,어? 이제 일어났다고? 어?,0
3,아까 잠깐 깼다가 또 잤다,우와... 대단하다 언제 잤길래,0
4,새벽 5시? 많이 잔 것도 아님,그렇게 따지고 보면 그렇네?,0


In [ ]:
labeling_data.shape

(470959, 3)

In [ ]:
# 빈값이 있는 데이터는 삭제(383개)
labeling_data = labeling_data[(labeling_data['Q']!='') & (labeling_data['A']!='')].reset_index(drop=True)
print(labeling_data.shape)

(470576, 3)


In [ ]:
def returnText(x): # 한글자가 아닌 것들 + 네 라는 대답이 아니면 반환
    if (len(x) < 2) and (x != '네'):
        return x
aa = labeling_data.A.apply(returnText)

In [ ]:
labeling_data = labeling_data[aa.isnull()].reset_index(drop=True)

In [ ]:
labeling_data.to_csv('train_data.csv', index=False)

In [ ]:
pd.read_csv('train_data.csv')

,Q,A,label
0,오빠야 저녁 식사는 뭐야?,나는 아직 뭐 먹을지 모르겠네 니는?,0
1,난 일단 지금 라면 먹고 있는데 저녁 모르겠어,오 지금 라면을 먹음 저녁 먹을 수 있나,0
2,모르겠어 이제 일어나서,어? 이제 일어났다고? 어?,0
3,아까 잠깐 깼다가 또 잤다,우와... 대단하다 언제 잤길래,0
4,새벽 5시? 많이 잔 것도 아님,그렇게 따지고 보면 그렇네?,0
...,...,...,...
469964,음 저는 받은 파일이 너무 힘이 드네요,빨리 끝내고 다른 거 하고 싶어요 ㅠㅠ,0
469965,어떤 점이 제일 힘드시나요?,말이 너무 많고 중간 중간에 많이 끼어드네요,0
469966,아이고 힘드시겠네요.,이전에는 어떤 회사를 다니셨어요?,0
469967,이 일을 하기 전에는 집에서 공부하면서 아르바이트 겸 택배를 했습니다,쿠팡 플렉스라고 아시나요?,0


# 존댓말로 변경(취소)
https://velog.io/@acdongpgm/NLP.-%ED%95%9C%EA%B5%AD%EC%96%B4Korean%EC%97%90%EC%84%9C-%EB%B0%98%EB%A7%90informal%EC%9D%84-%EC%A1%B4%EB%8C%93%EB%A7%90formal%EB%A1%9C-%EB%B0%94%EA%BF%94%EC%A3%BC%EB%8A%94-%EB%B3%80%ED%99%98%EA%B8%B0convertor-korean-Formal-Convertor-Using-Deep-Learning

적용해봤으나 너무 느림

In [ ]:
Chatbot_Data = pd.concat([Chatbot_Data, labeling_data])

In [ ]:
Chatbot_Data.shape

(497373, 3)

In [ ]:
import torch
from transformers import GPT2LMHeadModel
from transformers import PreTrainedTokenizerFast

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import re
from tqdm import tqdm
import urllib.request

In [ ]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK) 
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
Chatbot_Data = pd.read_csv('chatbot_data_240801_1.csv')
print(Chatbot_Data.shape)
Chatbot_Data.head()

(432634, 3)


,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


# 챗봇 학습 단계

In [39]:
# 챗봇 데이터를 처리하는 클래스를 만든다.
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=40):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = koGPT2_TOKENIZER

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn["Q"]  # 질문을 가져온다.
        q = re.sub(r"([?.!,])", r" ", q)  # 구둣점들을 제거한다.

        a = turn["A"]  # 답변을 가져온다.
        a = re.sub(r"([?.!,])", r" ", a)  # 구둣점들을 제거한다.

        q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        #질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.    
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)

In [40]:
learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 1,2회차는 epoch 30회 3회차는 10회, 4회차 20회, 5회차 10회
epoch_num = 10
Sneg = -1e18

In [41]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)
    # return torch.stack(data), torch.stack(mask), torch.stack(label)

In [42]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_set = ChatbotDataset(Chatbot_Data, max_len=40)
train_dataloader = DataLoader(train_set, batch_size=32, num_workers=0, shuffle=True, collate_fn=collate_batch, pin_memory=True)
# pin_memory를 이용하여 데이터를 고정된 주소에 로드
# 로컬에서는 큰 차이가 없었음;;

In [43]:
model.to(device)
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

https://jaeyung1001.tistory.com/entry/bf16-fp16-fp32%EC%9D%98-%EC%B0%A8%EC%9D%B4%EC%A0%90

In [44]:
# FP16(반정밀도) 연산 적용하여 학습 속도 향상(약2분30초 -> 약1분 30초)
from torch.cuda.amp import GradScaler, autocast

scaler = GradScaler()

for epoch in tqdm(range(epoch_num)):
    for batch_idx, samples in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        token_ids = token_ids.to(device)
        mask = mask.to(device)
        label = label.to(device)
        
        with autocast():
            out = model(token_ids)
            out = out.logits
            mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
            mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
            loss = criterion(mask_out.transpose(2, 1), label)
            avg_loss = loss.sum() / mask.sum()
        
        scaler.scale(avg_loss).backward()
        scaler.step(optimizer)
        scaler.update()

  0%|          | 0/10 [00:00<?, ?it/s]C:\Users\fkfma\AppData\Local\Temp\ipykernel_4884\4262151529.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:277.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
c:\Users\fkfma\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:650: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
100%|██████████| 10/10 [

In [ ]:
# from torch.cuda.amp import GradScaler, autocast

# scaler = GradScaler()

# for epoch in tqdm(range(epoch_num)):
#     total_loss = 0.0
#     for batch_idx, samples in enumerate(train_dataloader):
#         optimizer.zero_grad()
#         token_ids, mask, label = samples
#         token_ids = token_ids.to(device)
#         mask = mask.to(device)
#         label = label.to(device)
        
#         with autocast():
#             out = model(token_ids)
#             out = out.logits
#             mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
#             mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
#             loss = criterion(mask_out.transpose(2, 1), label)
#             avg_loss = loss.sum() / mask.sum()
        
#         scaler.scale(avg_loss).backward()
#         scaler.step(optimizer)
#         scaler.update()
        
#         total_loss += avg_loss.item()
    
#     print(f'Epoch {epoch+1}/{epoch_num}, Loss: {total_loss/len(train_dataloader)}')


In [ ]:
# print ("start")
# for epoch in tqdm(range(epoch_num)):
#     for batch_idx, samples in enumerate(train_dataloader):
#         optimizer.zero_grad()
#         token_ids, mask, label = samples
#         token_ids = token_ids.to(device)
#         mask = mask.to(device)
#         label = label.to(device)
#         out = model(token_ids)
#         out = out.logits      #Returns a new tensor with the logit of the elements of input
#         mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
#         mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
#         loss = criterion(mask_out.transpose(2, 1), label)
#         # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
#         avg_loss = loss.sum() / mask.sum()
#         avg_loss.backward()
#         # 학습 끝
#         optimizer.step()
# print ("end")

In [45]:
import os
model_save_path = "chatbot_model0801_432634.pth"
torch.save(
    {
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "epoch": epoch,
    },
    model_save_path,
)

# 바로 모델을 추론하는 경우 아래부터 진행

In [1]:
import torch
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK) 
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
model.to('cpu')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

# 모델 학습 완료 후 추론시 아래 코드부터 진행

In [2]:
model_path = "chatbot_model0801_432634.pth" 
# checkpoint = torch.load(model_path)
checkpoint = torch.load(model_path, map_location='cpu')
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [3]:
# 기존 인코딩 코드 주석 내용
# sent='0'
# Q_TKN + q + SENT + sent + A_TKN + a

# 아래 두 코드는 학습없이 추론만 진행시 활성화
# device = torch.device('cuda:0')
device = torch.device('cpu')
model.to(device)

with torch.no_grad():
    while True:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while True:
            input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + q + SENT + A_TKN + a)).unsqueeze(dim=0)
            # input_ids = input_ids.to(device)
            pred = model(input_ids)
            pred = pred.logits
            gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).to('cpu').squeeze().numpy().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        
        print("User > {}".format(q))
        print("Chatbot > {}".format(a.replace('<pad>', '').strip()))

User > 안녕
Chatbot > 안녕하세요
User > 넌 누구야?
Chatbot > 넌
User > 똑바로 대답해
Chatbot > 좋은 말만 해보세요
User > 재밌는 영화 추천해줘
Chatbot > 최신 영화가 좋을 것 같아요
User > 최신 영화말고 아는 거 있어?
Chatbot > 최신 영화
User > 서부 영화는 어때?
Chatbot > 서부 영화
User > 아침인데 습해
Chatbot > 감기 걸리기 딱 좋은 날씨에요
User > 너무 더워
Chatbot > 시원한 음료 한잔 드세요
User > 아무것도 하기 싫다
Chatbot > 저랑 이야기해요
User > 무슨 얘기할까?
Chatbot > 저는 위로해드리는 로봇이에요
User > 채팅이 안보여
Chatbot > 친구한테 부탁해보세요
User > 무슨 부탁을 해?
Chatbot > 저는 비밀을 보장해요
User > 너의 비밀이 뭐야?
Chatbot > 비밀은 비밀을 지키는 것이죠
User > 짱나네
Chatbot > 저도 궁금하네요
User > 짜증난다고
Chatbot > 짜증날 땐 짜장면


In [ ]:
gdgdg
ㅇㅇㅇ
배고파
오늘 날씨 어떨까?
영화 추천해줘
오늘 뭐할까?
아니 뭔소리야
ㅎㅇㅎㅇ
왤케 대답이 늦어
뭐라는 거야


ckao
Passw0rd

In [ ]:
with torch.no_grad():
    q = input("user > ").strip()
    a = ""
    while True:
        input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + q + SENT + A_TKN + a)).unsqueeze(dim=0)
        input_ids = input_ids.to(device)
        pred = model(input_ids)
        pred = pred.logits
        gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).to('cpu').squeeze().numpy().tolist())[-1]
        if gen == EOS:
            break
        a += gen.replace("▁", " ")
    
    print("User > {}".format(q))
    print("Chatbot > {}".format(a.replace('<pad>', '').strip()))

User > 배고프다
Chatbot > 오늘은 맛있는 걸로 든든하게 드세요


In [ ]:
# https://huggingface.co/datasets/MarkrAI/KoCommercial-Dataset?library=true # 여러 데이터셋 설명 주소
# https://huggingface.co/datasets/HumanF-MarkrAI/WIKI_QA_Near_dedup

In [ ]:
# 추가 데이터셋 탐색
# https://huggingface.co/datasets/jojo0217/korean_safe_conversation # 인공지능 대화 데이터셋(성균관대)
# 위 주소에 다양한 데이터셋 주소가 적혀있음
# https://huggingface.co/datasets/Bingsu/ko_alpaca_data 여기에는 input 열이 추가로 있음
# https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=data&dataSetSn=106
# 위 주소에서 일상대화 데이터셋 추가 적용예정
# https://huggingface.co/datasets/JosephLee/science_textbook_elementary_kor_seed 과학
# https://www.aihub.or.kr/aihubdata/data/dwld.do?currMenu=115&topMenu=100 감성 대화 말뭉치
https://huggingface.co/datasets/heegyu/OIG-small-chip2-ko 
# 

In [ ]:
https://huggingface.co/datasets/CarrotAI/ko-code-alpaca-QA?row=87 장문의 대답(정제 필요 = 영문 대답, 토큰 개수에 따른 처리 필요)
3회차 데이터는 _20으로 끝나는 파일만 학습하면 만화 및 영화 관련 내용만 학습이 가능함

In [ ]:
요약데이터셋
https://huggingface.co/datasets/daekeun-ml/naver-news-summarization-ko